In [1]:
import json
import csv
import pandas as pd
import numpy as np
import os
import jieba
import re

In [2]:
# 1. PRE-PROCESSING

# Read in input file 
my_var = open('haodfcomments.csv').read().split()


In [3]:
# Display first 10 entries of input
print(my_var[:10])


['评论内容,,,,', '张主任很耐心，医者仁心,,,,', '个人觉得,,,,', '张主任医术高明，非常感谢张主任，耐心的为我分析病情的情况，感谢您张主任,,,,', '周主任不仅人长的漂亮，心地还特别善良，对患者很有耐心，认真负责，医术精湛。,,,,', '周医生很细心，很负责任,,,,', '周医生技术精湛，是送子观音，又能为患者节约，真是最美的白衣天使,,,,', '周医生对待病人热情耐心，非常感谢周医生，给病人解除了痛苦，治好了我多年的耳鸣，眩晕症！,,,,', '起床眩晕，医生说耳石症，复位好了。谢谢,,,,', '态度和蔼！工作认真仔细！是我心中的好医生！就是排队时间太久，不过总体还是不错的！,,,,']


In [4]:
# Select stopwords (words lacking significance to strip from input)
stopwords = [i.strip() for i in open('./baidu1.txt', "r", encoding = 
                                     "utf-8").readlines()]


In [5]:
# Function to strip stopwords
def pretty_cut(sentence):
    cut_list = jieba.lcut(''.join(re.findall('[\u4e00-\u9fa5]', sentence)), cut_all=True)
    for i in range(len(cut_list) - 1, -1, -1):
        if cut_list[i] in stopwords:
            del cut_list[i]
    return cut_list


In [6]:
# Write stripped input to new csv file hello1
with open(os.path.join("./hello1.csv"), "w", encoding = "utf-8", newline = '') as g:
    writer = csv.writer(g)
    writer.writerow(["content"])
    for line in my_var:
        content = line.strip(" ")
        cut_content = " ".join(pretty_cut(content))
        writer.writerow([cut_content])


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/pq/cl_1w39d17v1d0myh0903sxm0000gn/T/jieba.cache
Loading model cost 0.288 seconds.
Prefix dict has been built successfully.


In [9]:
input_file = pd.read_csv('hello1.csv', usecols = ['content'])
input_file = input_file.fillna('')
input_file = input_file.values.tolist()

from nltk import flatten
my_input = flatten(input_file)

In [8]:
# 2. BERTopic TRANSFORMATIONS

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(tokenizer=jieba.lcut)


In [10]:
X = count_vectorizer.fit_transform(my_input)


/Users/hannahsun/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
count_vectorizer.get_feature_names_out()

array([' ', '一', '一一', ..., '龙海', '龚', '龟头'], dtype=object)

In [15]:
pd.DataFrame(X.toarray(), columns = count_vectorizer.get_feature_names_out())

,,一,一一,一万,一万个,一万元,一万四千,一万块,一万多,一万多块,...,龋,龋齿,龙,龙凤,龙凤胎,龙大,龙江,龙海,龚,龟头
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29226,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29227,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29228,35,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29229,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
print(count_vectorizer.get_feature_names_out())

[' ' '一' '一一' ... '龙海' '龚' '龟头']


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
def tokenize(sentence):
    without_duplicates = re.sub(r'(.)\1+', r'\1\1', sentence)
    without_punctuation = re.sub(r'[^\w]','',without_duplicates)
    return jieba.lcut(without_duplicates)

In [22]:
l2_vectorizer = TfidfVectorizer(tokenizer=tokenize, use_idf=True)
X = l2_vectorizer.fit_transform(my_input)
l2_df = pd.DataFrame(X.toarray(), columns=l2_vectorizer.get_feature_names_out())

/Users/hannahsun/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [26]:
vectorizer = TfidfVectorizer(use_idf=True, tokenizer=tokenize)
X = vectorizer.fit_transform(my_input)

/Users/hannahsun/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [31]:
from sklearn.cluster import KMeans

number_of_clusters = 300
km = KMeans(n_clusters=number_of_clusters)
km.fit(X)

/Users/hannahsun/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=300)

In [32]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Top terms per cluster:
Cluster 0: 解释   清楚 详细 耐心
Cluster 1:   毛 手术 微创 主任
Cluster 2:   重要 最重 重 很
Cluster 3:   袁 谢谢 关心 高尚
Cluster 4: 态度 医生   如花似玉 如约
Cluster 5:   张 主任 医生 非常
Cluster 6:   确诊 明确 诊断 治疗
Cluster 7:   有问必答 必答 医生 病人
Cluster 8:   解惑 答疑 业务精 非常感谢
Cluster 9: 负责   病人 医生 态度
Cluster 10: 生态   态度 医生 医术
Cluster 11: 耐心   病人 医生 态度
Cluster 12: 比较   耐心 医生 负责
Cluster 13:   德 医德 高尚 高
Cluster 14:   药 开 吃 医生
Cluster 15:   主任 医生 年月日 您好
Cluster 16: 信赖 值得   医生 医术
Cluster 17:   高明 认真负责 认真 负责
Cluster 18:   赵 主任 医生 感谢
Cluster 19:   认真负责 认真 负责 态度
Cluster 20:   很 细 细心 医生
Cluster 21: 敬业   专业 医生 非常
Cluster 22:   妙手仁心 妙手 仁心 德
Cluster 23:   陈医生 医生 感谢 非常
Cluster 24: 技术   精湛 态度 高
Cluster 25: 王   主任 感谢 医术
Cluster 26: 亲和力   强 医生 老师
Cluster 27:   解决 问题 解决问题 医生
Cluster 28: 感谢您   感谢 非常感谢 非常
Cluster 29:   赞 点 医生 生
Cluster 30:   精湛 医术 态度 和蔼
Cluster 31:   高明 德 医德 高尚
Cluster 32:   陈医生 生态 态度 医生
Cluster 33:   德 高尚 医德 高
Cluster 34:   朱 晨 主任 感谢
Cluster 35:   一丝不苟 不苟 一丝 工作
Cluster 36:   和蔼 可亲 和蔼可亲 态度
Cluster 37:   许 医生 主任 医术

In [33]:
results = pd.DataFrame()
results['text'] = my_input
results['category'] = km.labels_
results

,text,category
0,评论 内容,15
1,张 主任 耐心 医者 仁心,107
2,个人 觉得,15
3,张 主任 医术 高明 非常 非常感谢 感谢 张 主任 耐心 分析 病情 情况 感谢 感谢您 ...,136
4,周 主任 长 漂亮 心地 特别 善良 患者 耐心 认真 认真负责 负责 医术 精湛,53
...,...,...
29226,医生 认真 认真负责 负责 仔细 做 体格 体格检查 检查 没有 重复 化验 检查 节约 支...,53
29227,周 主任 任人 耐心 心细 细心 诊断 断后 给出 医疗 建议 医德 医术 高明 医生 四 ...,191
29228,和蔼 和蔼可亲 可亲 思路 思路清晰 清晰 看见 动态 血压 报告 主动 测量 三次 血压 ...,191
29229,态度 水平 高 治好 多年 心脏 心脏病 全家 十分 十分满意 满意,188
